In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, auc
from keras.models import Sequential, Model
from keras.layers import Dense, Input
from keras.layers.normalization import BatchNormalization
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from keras.utils.np_utils import to_categorical
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder
from keras import metrics

Using TensorFlow backend.


In [5]:
os.getcwd()
textdata = pd.read_csv("train-2.csv")
textdata.shape
textdata.head()
textdata.columns
textdata.index
textdata.describe(include='all')
textdata.dtypes
textdata.drop('fileid', axis=1, inplace=True)
textdata['categories'] = textdata['categories'].astype('category')
textdata.shape
print(textdata.isnull().sum())

converse      24
categories     0
dtype: int64


In [6]:
textdata.dtypes

converse        object
categories    category
dtype: object

In [7]:
max_num = 3500
tags = textdata['categories'].astype(str)
texts = textdata['converse'].astype(str)
le = LabelEncoder()
tags = le.fit_transform(tags)
tok = Tokenizer(num_words = max_num)
tok.fit_on_texts(texts)
mat_texts = tok.texts_to_matrix(texts, mode='count')
print(tags[:5])
print(mat_texts[:5])
print(tags.shape,mat_texts.shape)

[0 0 0 5 1]
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  3.  0. ...,  0.  0.  0.]
 [ 0.  2.  2. ...,  0.  0.  0.]
 [ 0.  2.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]]
(31350,) (31350, 3500)


In [8]:
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
onehot_encoder = OneHotEncoder(sparse=False)
tags = tags.reshape(len(tags),1)
y = onehot_encoder.fit_transform(tags)
X = mat_texts[:]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=123)

In [9]:
print(y)

[[ 1.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.]
 ..., 
 [ 0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  1.]]


In [11]:
model = Sequential()
model.add(Dense(1000, activation='relu', kernel_initializer='random_uniform', input_shape=(max_num,)))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(250, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.1))
model.add(BatchNormalization())
model.add(Dense(6, activation='softmax'))
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[metrics.categorical_accuracy])
model.fit(X_train, y_train, batch_size= 1024, epochs=25, validation_split=0.25)

TypeError: `Dense` can accept only 1 positional arguments ('units',), but you passed the following positional arguments: [1000, 8]

In [22]:
train_pred = model.predict_classes(X_train)
train_pred
test_pred = model.predict_classes(X_test)
print(test_pred)

[1 5 4 ..., 1 0 4]


In [23]:
train_pred = model.predict_classes(X_train)
#train_pred = pd.get_dummies(train_pred)
print(train_pred)
#print(metrics.confusion_matrix(y_train.flatten(), train_pred.flatten()))
#y_test_non_category = [ np.argmax(t) for t in y_test ]
#y_predict_non_category = [ np.argmax(t) for t in train_pred ]
#conf_mat = confusion_matrix(y_test_non_category, y_predict_non_category)
##accuracy_Train_hiddenExp = (confusion_matrix_train[0,0]+confusion_matrix_train[1,1])/(confusion_matrix_train[0,0]+confusion_matrix_train[0,1]+confusion_matrix_train[1,0]+confusion_matrix_train[1,1])
#specificity_Train_hiddenExp = confusion_matrix_train[0,0]/(confusion_matrix_train[0,0]+confusion_matrix_train[0,1])
#recall_Train_hiddenExp = confusion_matrix_train[1,1]/(confusion_matrix_train[1,0]+confusion_matrix_train[1,1])
#precision_Train_hiddenExp = confusion_matrix_train[1,1]/(confusion_matrix_train[0,1]+confusion_matrix_train[1,1])
#print(confusion_matrix_train,accuracy_Train_hiddenExp,specificity_Train_hiddenExp,recall_Train_hiddenExp,precision_Train_hiddenExp)


[1 0 1 ..., 3 4 4]


In [24]:
test_pred = model.predict_classes(X_test)
print(test_pred)
#confusion_matrix_test = confusion_matrix(y_test, test_pred)
#accuracy_Test_hiddenExp = (confusion_matrix_test[0,0]+confusion_matrix_test[1,1])/(confusion_matrix_test[0,0]+confusion_matrix_test[0,1]+confusion_matrix_test[1,0]+confusion_matrix_test[1,1])
#specificity_Test_hiddenExp = confusion_matrix_test[0,0]/(confusion_matrix_test[0,0]+confusion_matrix_test[0,1])
#recall_Test_hiddenExp = confusion_matrix_test[1,1]/(confusion_matrix_test[1,0]+confusion_matrix_test[1,1])
#precision_Test_hiddenExp = confusion_matrix_test[1,1]/(confusion_matrix_test[0,1]+confusion_matrix_test[1,1])
#print(confusion_matrix_test,accuracy_Test_hiddenExp,specificity_Test_hiddenExp,recall_Test_hiddenExp,precision_Test_hiddenExp)

[1 5 4 ..., 1 0 4]


In [26]:
import csv,sys
graderdata = pd.read_csv("test-2.csv")
graderdata.shape
graderdata.head()
graderdata.drop('fileid', axis=1, inplace=True)
graderdata.shape
max_num = 3500
texts_new = graderdata['converse'].astype(str)
tok = Tokenizer(num_words = max_num)
tok.fit_on_texts(texts_new)
mat_texts_new = tok.texts_to_matrix(texts_new, mode='count')
print(mat_texts_new[:5])
print(mat_texts_new.shape)
grader_pred = model.predict_classes(mat_texts_new)
print(grader_pred[:5])
os.getcwd()
my_dfs = pd.DataFrame(grader_pred)
my_dfs.to_csv('out.csv', index=False, header=False)
pred_data = pd.read_csv("out.csv")
inverted = le.inverse_transform(pred_data)
print(inverted)

[[ 0.  2.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  1.  2. ...,  0.  0.  0.]
 [ 0.  0.  3. ...,  0.  0.  0.]
 [ 0.  0.  4. ...,  0.  0.  0.]]
(11456, 1500)
[1 0 1 0 1]
[['APPOINTMENTS']
 ['ASK_A_DOCTOR']
 ['APPOINTMENTS']
 ..., 
 ['APPOINTMENTS']
 ['APPOINTMENTS']
 ['ASK_A_DOCTOR']]


In [27]:
grader_data = pd.DataFrame(inverted)
grader_data.to_csv('graddata.csv', index=False, header=False)
os.getcwd()

'/Users/Downloads'